In [1]:
# import packages
!pip install faker
!pip install names
!pip uninstall -y dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl
!pip install dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl

import json
import pandas
import faker
import os
from IPython.display import display
import requests
import random
import ipywidgets as wid

Uninstalling OpenSpecimenAPIconnector-0.9.1:
  Successfully uninstalled OpenSpecimenAPIconnector-0.9.1
Processing ./dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl


In [2]:
import OpenSpecimenAPIconnector.os_core as os_core
import OpenSpecimenAPIconnector.os_util as os_util

In [3]:
def csv_download_link(df, csv_file_name, delete_prompt=False):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    from IPython.core.display import display, HTML
    link = "http://localhost:8888/edit/work/" + csv_file_name
    df.to_csv(csv_file_name, index=False)
    display(HTML("<a href={} target=_blank>Download/Edit</a>".format(link)))
   
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

In [4]:
def multi_checkbox_widget(options_dict):
    """ Widget with a search field and lots of checkboxes """
    search_widget = wid.Text()
    output_widget = wid.Output()
    options = [x for x in options_dict.values()]
    options_layout = wid.Layout(
        overflow='auto',
        border='1px solid black',
        width='300px',
        height='300px',
        flex_flow='column',
        display='flex'
    )
    
    #selected_widget = wid.Box(children=[options[0]])
    options_widget = wid.VBox(options, layout=options_layout)
    #left_widget = wid.VBox(search_widget, selected_widget)
    multi_select = wid.VBox([search_widget, options_widget])

    @output_widget.capture()
    def on_checkbox_change(change):
        
        selected_recipe = change["owner"].description
        #print(options_widget.children)
        #selected_item = wid.Button(description = change["new"])
        #selected_widget.children = [] #selected_widget.children + [selected_item]
        options_widget.children = sorted([x for x in options_widget.children], key = lambda x: x.value, reverse = True)
        
    for checkbox in options:
        checkbox.observe(on_checkbox_change, names="value")

    # Wire the search field to the checkboxes
    @output_widget.capture()
    def on_text_change(change):
        search_input = change['new']
        if search_input == '':
            # Reset search field
            new_options = sorted(options, key = lambda x: x.value, reverse = True)
        else:
            # Filter by search field using difflib.
            #close_matches = difflib.get_close_matches(search_input, list(options_dict.keys()), cutoff=0.0)
            close_matches = [x for x in list(options_dict.keys()) if str.lower(search_input.strip('')) in str.lower(x)]
            new_options = sorted(
                [x for x in options if x.description in close_matches], 
                key = lambda x: x.value, reverse = True
            ) #[options_dict[x] for x in close_matches]
        options_widget.children = new_options

    search_widget.observe(on_text_change, names='value')
    display(output_widget)
    return multi_select

In [5]:
# The URL of the Openspecimen
print('Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter')
#base_url='http://localhost:9000' # input() set to hardcoded at the moment just change below or add urls for more instances
base_url='http://biobank.silicolab.bibbox.org'
base_url2= 'http://biobank-7-2.silicolab.bibbox.org' 
base_url+='/openspecimen/rest/ng'
base_url2+='/openspecimen/rest/ng'
print('The whole URL for API Calls is: ' + base_url)

Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter
The whole URL for API Calls is: http://biobank.silicolab.bibbox.org/openspecimen/rest/ng


In [6]:
# Loginname and Pasword of the API-User, recommended is an extra API User
print('Loginname of the user:')
loginname='admin'#input() 
print('Password of the user:')
password='Login@123'#input()

Loginname of the user:
Password of the user:


The next step is to choose the entity which will be created or updated. The following schemas are in Openspecimen available: (Name, openspecimen entity). One have to write the openspecimen entity. ('Collection Protocol', 'cp') ('Collection Protocol Registration', 'cpr') ('User', 'user') ('User Roles', 'userRoles') ('Site', 'site') ('Shipments', 'shipment') ('Institute', 'institute') ('Distribution Protocol Requirement', 'dpRequirement') ('Distribution Protocol', 'distributionProtocol') ('Distribution Order', 'distributionOrder') ('Containers', 'storageContainer') ('Container Type', 'storagecontainerType') ('Container Shipment', 'containerShipment') ('Collection Protocol Event', 'cpe') ('Master Specimen', 'masterSpecimen') ('Participant', 'participant') ('Specimen Requirement', 'sr') ('Visit', 'visit') ('Specimen Aliquot', 'specimenAliquot') ('Specimen Derivate', 'specimenDerivative') ('Specimen Disposal', 'specimenDisposal') ('Consent', 'consent')

In [2]:
schema=widgets.Dropdown(
    options= [(), ('Collection Protocol', 'cp'), ('Specimen', 'specimen'), ('Collection Protocol Registration', 'cpr'),
              ('User', 'user'), ('User Roles', 'userRoles'), ('Site', 'site'), ('Shipments', 'shipment'), 
              ('Institute', 'institute'), ('Distribution Protocol Requirement', 'dpRequirement'), 
              ('Distribution Protocol', 'distributionProtocol'), ('Distribution Order', 'distributionOrder'), 
              ('Containers', 'storageContainer'), ('Container Type', 'storagecontainerType'),
              ('Container Shipment', 'containerShipment'), ('Collection Protocol Event', 'cpe'),
              ('Master Specimen', 'masterSpecimen'), ('Participant', 'participant'), ('Specimen Requirement', 'sr'),
              ('Visit', 'visit'), ('Specimen Aliquot', 'specimenAliquot'), 
              ('Specimen Derivate', 'specimenDerivative'),('Specimen Disposal', 'specimenDisposal'),
              ('Consent', 'consent')],
    value=('Specimen', 'specimen'),
    description='Entity:')

operation = widgets.Dropdown(
    options = ["CREATE",  'UPDATE'],
              value="CREATE",
              description='Operation:')

print('Choose your Entity:')
display(schema)
print('Choose Operation:')
display(operation)

Choose your Entity:


Dropdown(description='Entity:', index=2, options=((), ('Collection Protocol', 'cp'), ('Specimen', 'specimen'),…

Choose Operation:


Dropdown(description='Operation:', options=('CREATE', 'UPDATE'), value='CREATE')

In [35]:
#initialize csv_bulk, bulk_operations and CSV_exporter

csv_files= os_core.csv_bulk(base_url2,(loginname, password))
bulk_op= os_util.bulk_operations(base_url2, (loginname, password))
csv_export_7_2 = os_util.Export_OP(base_url2, (loginname, password))

In [30]:
# get necessary cp information

cp_tools = os_core.collection_protocol(base_url2, (loginname, password))

# get cp info and wirte a dict in order to simplify the api output
cps = cp_tools.get_all_collection_protocols()
cps_dict = {}
for i, item in enumerate(cps):
    
    print("Cp Title:")
    print(item["title"])
    print("Cp ID: ",item["id"], "\n")
    cps_dict[item["title"]] = item["id"]

#choose target CP / Check if matching is correct
#print("Target CP: ", cps_dict[0])
#cp_id_target = cps_dict[0][1]
#cp_target = cp_tools.get_collection_protocol(cp_id_target)
#cp_def_target = cp_tools.get_cp_def(cp_id_target)
#mult_cp.append([cp_id_target, cp_def_target])

Cp Title:
Main collection of Clinical-biological Biobank at INAB, comprising of all the samples.
Cp ID:  16 

Cp Title:
Hellenic Biobank of Overweight and Obesity in Childhood and Adolescence
Cp ID:  17 

Cp Title:
Hellenic Biobank for Parkinson's Disease
Cp ID:  18 



In [31]:
# Get lots of words for our options
#words_url = 'https://svnweb.freebsd.org/csrg/share/dict/words?view=co&content-type=text/plain/'
#response = requests.get(words_url)
#print(response)
#input()
#response.raise_for_status()
#words = ["test", "penis", "mama", "papa", "kind"]

#words = set([word.lower() for word in words.splitlines()])
descriptions = [key for key in  cps_dict.keys()]

options_dict = {
    x: wid.Checkbox(
        description=x, 
        value=False,
        style={"description_width":"0px"}
    ) for x in descriptions
}

def f(**args):
    
    results = [key for key, value in args.items() if value]
    display(results)

ui = multi_checkbox_widget(options_dict)
out = wid.interactive_output(f, options_dict)
display(wid.HBox([ui, out]))

Output()

In [38]:
# get selected cp_ids
cp_ids = []
selected_options = [widget.description for widget in ui.children[1].children if widget.value]
for item in selected_options:
    cp_ids.append(cps_dict[item])
    
print(cp_ids)

[17]
